In [1]:
import pandas as pd
import numpy as np

In [62]:
# Load the time series data
junc1 = pd.read_csv('Traffic_Junc/Junc1.csv')

In [64]:
junc1.head()

,DateTime,Junction,Vehicles
DateTime,,,
2015-11-01 00:00:00,2015-11-01 00:00:00,1,15
2015-11-01 01:00:00,2015-11-01 01:00:00,1,13
2015-11-01 02:00:00,2015-11-01 02:00:00,1,10
2015-11-01 03:00:00,2015-11-01 03:00:00,1,7
2015-11-01 04:00:00,2015-11-01 04:00:00,1,9


### Predictions

In [65]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta

In [70]:
# Preprocess the data
scaler = MinMaxScaler()
junc1['scaled_traffic'] = scaler.fit_transform(junc1['Vehicles'].values.reshape(-1, 1))

junc1.head()

,DateTime,Junction,Vehicles,scaled_traffic
DateTime,,,,
2015-11-01 00:00:00,2015-11-01 00:00:00,1,15,0.066225
2015-11-01 01:00:00,2015-11-01 01:00:00,1,13,0.052980
2015-11-01 02:00:00,2015-11-01 02:00:00,1,10,0.033113
2015-11-01 03:00:00,2015-11-01 03:00:00,1,7,0.013245
2015-11-01 04:00:00,2015-11-01 04:00:00,1,9,0.026490


In [75]:
# Split the data into training and testing sets
train_size = int(len(data) * 0.8)
train_data = junc1.iloc[:train_size]
test_data = junc1.iloc[train_size:]

train_data

,DateTime,Junction,Vehicles,scaled_traffic
DateTime,,,,
2015-11-01 00:00:00,2015-11-01 00:00:00,1,15,0.066225
2015-11-01 01:00:00,2015-11-01 01:00:00,1,13,0.052980
2015-11-01 02:00:00,2015-11-01 02:00:00,1,10,0.033113
2015-11-01 03:00:00,2015-11-01 03:00:00,1,7,0.013245
2015-11-01 04:00:00,2015-11-01 04:00:00,1,9,0.026490
...,...,...,...,...
2017-03-01 04:00:00,2017-03-01 04:00:00,1,42,0.245033
2017-03-01 05:00:00,2017-03-01 05:00:00,1,39,0.225166
2017-03-01 06:00:00,2017-03-01 06:00:00,1,43,0.251656


In [76]:
# Create input sequences and target values
sequence_length = 10

def create_sequences(data, sequence_length):
    x = []
    y = []
    for i in range(len(data) - sequence_length):
        x.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(x), np.array(y)

In [77]:
train_x, train_y = create_sequences(train_data['scaled_traffic'].values, sequence_length)
test_x, test_y = create_sequences(test_data['scaled_traffic'].values, sequence_length)

In [78]:
# Flatten the input sequences
train_x = train_x.reshape(train_x.shape[0], -1)
test_x = test_x.reshape(test_x.shape[0], -1)


In [79]:
# Build the linear regression model
model = LinearRegression()

# Train the model
model.fit(train_x, train_y)

LinearRegression()

In [80]:
# Evaluate the model
train_predictions = model.predict(train_x)
train_rmse = np.sqrt(mean_squared_error(train_y, train_predictions))
print('Train RMSE:', train_rmse)

test_predictions = model.predict(test_x)
test_rmse = np.sqrt(mean_squared_error(test_y, test_predictions))
print('Test RMSE:', test_rmse)

Train RMSE: 0.0351082728594234
Test RMSE: 0.04779015272503905


In [82]:
# Predict traffic for a future time
current_time = datetime.now()
future_time = current_time + timedelta(hours=1)
future_time_str = future_time.strftime('%Y-%m-%d %H:%M:%S')

predicted_traffic = model.predict(test_x[-1].reshape(1, -1))
predicted_traffic = scaler.inverse_transform(predicted_traffic.reshape(1, -1))

print('Current Time:', current_time)
print('Future Time:', future_time_str)
print('Predicted Traffic at Future Time:', predicted_traffic[0][0])


Current Time: 2023-07-04 16:00:28.793935
Future Time: 2023-07-04 17:00:28
Predicted Traffic at Future Time: 81.00458447195213
